In [1]:
print("ok   ")

ok   


LLM Model - Google Gemini

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
model=ChatGoogleGenerativeAI(model='gemini-1.5-flash')
output=model.invoke("hi")
print(output.content)

Hi there! How can I help you today?


Embedding Model - HuggingFace

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en")
len(embeddings.embed_query("hi"))

c:\hk\agentic2.0\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


384

Import Data and Store in Vector DB - save it on Disk

In [4]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [9]:
loader=DirectoryLoader("ncsdata",glob="./*.txt",loader_cls=TextLoader)

In [10]:
docs=loader.load()

In [11]:
docs

[Document(metadata={'source': 'ncsdata\\ncs.txt'}, page_content='=== NCS Department Contact Directory ===\n\nHuman Resources: hr@au.ncs.co\nTimesheet Support: timesheet@au.ncs.co\nAccounts Payable: accounts@au.ncs.co\nBilling Inquiries: billing@au.ncs.co\nIT Service Desk: servicedesk@au.ncs.co\nCloud Security: security@au.ncs.co\nReception: reception@au.ncs.co\nProject Management Office: pmo@au.ncs.co\nRecruitment Team: recruitment@au.ncs.co\nLegal & Compliance: legal@au.ncs.co')]

In [12]:
docs[0].page_content

'=== NCS Department Contact Directory ===\n\nHuman Resources: hr@au.ncs.co\nTimesheet Support: timesheet@au.ncs.co\nAccounts Payable: accounts@au.ncs.co\nBilling Inquiries: billing@au.ncs.co\nIT Service Desk: servicedesk@au.ncs.co\nCloud Security: security@au.ncs.co\nReception: reception@au.ncs.co\nProject Management Office: pmo@au.ncs.co\nRecruitment Team: recruitment@au.ncs.co\nLegal & Compliance: legal@au.ncs.co'

In [16]:
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=5
)

In [17]:
new_docs=text_splitter.split_documents(documents=docs)

In [18]:
new_docs

[Document(metadata={'source': 'ncsdata\\ncs.txt'}, page_content='=== NCS Department Contact Directory ==='),
 Document(metadata={'source': 'ncsdata\\ncs.txt'}, page_content='Human Resources: hr@au.ncs.co'),
 Document(metadata={'source': 'ncsdata\\ncs.txt'}, page_content='Timesheet Support: timesheet@au.ncs.co'),
 Document(metadata={'source': 'ncsdata\\ncs.txt'}, page_content='Accounts Payable: accounts@au.ncs.co'),
 Document(metadata={'source': 'ncsdata\\ncs.txt'}, page_content='Billing Inquiries: billing@au.ncs.co'),
 Document(metadata={'source': 'ncsdata\\ncs.txt'}, page_content='IT Service Desk: servicedesk@au.ncs.co'),
 Document(metadata={'source': 'ncsdata\\ncs.txt'}, page_content='Cloud Security: security@au.ncs.co'),
 Document(metadata={'source': 'ncsdata\\ncs.txt'}, page_content='Reception: reception@au.ncs.co'),
 Document(metadata={'source': 'ncsdata\\ncs.txt'}, page_content='Project Management Office: pmo@au.ncs.co'),
 Document(metadata={'source': 'ncsdata\\ncs.txt'}, page_co

In [19]:
doc_string=[doc.page_content for doc in new_docs]

In [20]:
doc_string

['=== NCS Department Contact Directory ===',
 'Human Resources: hr@au.ncs.co',
 'Timesheet Support: timesheet@au.ncs.co',
 'Accounts Payable: accounts@au.ncs.co',
 'Billing Inquiries: billing@au.ncs.co',
 'IT Service Desk: servicedesk@au.ncs.co',
 'Cloud Security: security@au.ncs.co',
 'Reception: reception@au.ncs.co',
 'Project Management Office: pmo@au.ncs.co',
 'Recruitment Team: recruitment@au.ncs.co',
 'Legal & Compliance: legal@au.ncs.co']

In [21]:
len(doc_string)

11

In [22]:
db=Chroma.from_documents(new_docs,embeddings)

In [23]:
retriever=db.as_retriever(search_kwargs={"k": 3})

In [24]:
retriever.invoke("Whats email address of HR")

[Document(metadata={'source': 'ncsdata\\ncs.txt'}, page_content='Human Resources: hr@au.ncs.co'),
 Document(metadata={'source': 'ncsdata\\ncs.txt'}, page_content='IT Service Desk: servicedesk@au.ncs.co'),
 Document(metadata={'source': 'ncsdata\\ncs.txt'}, page_content='Timesheet Support: timesheet@au.ncs.co')]

## creation of pydantic class

In [25]:
import operator
from typing import List
from pydantic import BaseModel , Field
from langchain.prompts import PromptTemplate
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph,END

In [26]:
class TopicSelectionParser(BaseModel):
    Topic:str=Field(description="selected topic")
    Reasoning:str=Field(description='Reasoning behind topic selection')

In [27]:
from langchain.output_parsers import PydanticOutputParser

In [28]:
parser=PydanticOutputParser(pydantic_object=TopicSelectionParser)

In [29]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"Topic": {"description": "selected topic", "title": "Topic", "type": "string"}, "Reasoning": {"description": "Reasoning behind topic selection", "title": "Reasoning", "type": "string"}}, "required": ["Topic", "Reasoning"]}\n```'

### this agentstate class you need to inside the stategraph

In [30]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [31]:
state={"messages":["hi"]}

In [32]:
state="hi"

In [33]:
def function_1(state:AgentState):
    
    question=state["messages"][-1]
    
    print("Question",question)
    
    template="""
    Your task is to classify the given user query into one of the following categories: [LLM,Internal,Web Related]. 
    Only respond with the category name and nothing else.

    User query: {question}
    {format_instructions}
    """
    
    prompt= PromptTemplate(
        template=template,
        input_variable=["question"],
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )
    
    
    chain= prompt | model | parser
    
    response = chain.invoke({"question":question})
    
    print("Parsed response:", response)
    
    return {"messages": [response.Topic]}

In [34]:
state={"messages":["what is a today weather?"]}

In [35]:
function_1(state)

Question what is a today weather?
Parsed response: Topic='Web Related' Reasoning='The query is a request for current weather information, typically obtained from a weather website or app.'


{'messages': ['Web Related']}

In [38]:
state={"messages":["What is NCS finance email address?"]}

In [39]:
function_1(state)

Question What is NCS finance email address?
Parsed response: Topic='Internal' Reasoning="The query asks for an internal email address, suggesting it's related to an internal system or organization."


{'messages': ['Internal']}

In [40]:
class TopicSelectionParser(BaseModel):
    Topic:str=Field(description="selected topic")
    Reasoning:str=Field(description='Reasoning behind topic selection')

In [41]:
def router(state:AgentState):
    print("-> ROUTER ->")
    
    last_message=state["messages"][-1]
    print("last_message:", last_message)
    
    if "Internal" in last_message.lower():
        return "RAG Call"
    elif "Web" in last_message.lower():
        return "Web Crawller Call"
    else:
        return "LLM Call"

In [42]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [43]:
# RAG Function
def function_2(state:AgentState):
    print("-> RAG Call ->")
    
    question = state["messages"][0]
    
    prompt=PromptTemplate(
        template="""You are NCS's internal assistant chatbot. Use the following context retrieved from internal systems to answer the staff query.
If the answer is not available, respond with "I don't know." Keep your response brief—no more than three sentences.\nQuestion: {question} \nContext: {context} \nAnswer:""",
        
        input_variables=['context', 'question']
    )
    
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
    )
    result = rag_chain.invoke(question)
    return  {"messages": [result]}

In [44]:
# LLM Function
def function_3(state:AgentState):
    print("-> LLM Call ->")
    question = state["messages"][0]
    
    # Normal LLM call
    complete_query = "Anwer the follow question with you knowledge of the real world. Following is the user question: " + question
    response = model.invoke(complete_query)
    return {"messages": [response.content]}

In [45]:
# Web Crawler Function
def function_4(state:AgentState):
    pass

In [46]:
from langgraph.graph import StateGraph,END

In [47]:
workflow=StateGraph(AgentState)

In [48]:
workflow.add_node("Supervisor",function_1)

In [49]:
workflow.add_node("RAG",function_2)

In [50]:
workflow.add_node("LLM",function_3)

In [52]:
workflow.add_node("WEB",function_4)

In [53]:
workflow.set_entry_point("Supervisor")

In [54]:
workflow.add_conditional_edges(
    "Supervisor",
    router,
    {
        "Internal": "RAG",
        "Web"     : "WEB",
        "LLM Call": "LLM",
    }
)

In [55]:
workflow.add_edge("RAG",END)
workflow.add_edge("LLM",END)
workflow.add_edge("WEB",END)

In [56]:
app=workflow.compile()

In [57]:
state={"messages":["hi"]}

In [58]:
app.invoke(state)

Question hi
Parsed response: Topic='Internal' Reasoning="The query 'hi' is too vague and doesn't relate to a specific LLM task or web-related request. It's likely an internal interaction or a simple greeting."
-> ROUTER ->
last_message: Internal
-> LLM Call ->


{'messages': ['hi', 'Internal', 'Hi there!']}

In [61]:
state={"messages":["I am working with NCS. Whats email address of NCS Finance"]}

In [62]:
app.invoke(state)

Question I am working with NCS. Whats email address of NCS Finance
Parsed response: Topic='Internal' Reasoning='The query is asking for an internal contact email address within a company (NCS). This is not a general web query or related to a large language model.'
-> ROUTER ->
last_message: Internal
-> LLM Call ->


{'messages': ['I am working with NCS. Whats email address of NCS Finance',
  'Internal',
  'I cannot provide you with the email address of NCS Finance.  This is because:\n\n* **Data Privacy:**  Contact information for specific departments within large companies like NCS is generally not publicly available to protect the privacy of employees and prevent potential phishing or spam attacks.\n* **Security:** Sharing such information publicly would be a security risk.\n\nTo obtain the correct email address, you should:\n\n* **Check NCS\'s website:** Look for a "Contact Us" section or a directory of departments.  They may list a general finance email address or a way to contact someone who can direct your inquiry.\n* **Use your company\'s internal directory (if applicable):** If you\'re already working with NCS, your company\'s internal directory likely has contact information for NCS employees.\n* **Contact your NCS point of contact:** If you have a specific person you\'re working with at N

In [63]:
state={"messages":["can you share news on Ahmedabad flight crash?"]}

In [64]:
result=app.invoke(state)

Question can you share news on Ahmedabad flight crash?
Parsed response: Topic='Web Related' Reasoning='The query requests news on a specific event, which is information typically found on the web.'
-> ROUTER ->
last_message: Web Related
-> LLM Call ->
